In [4]:
import pandas as pd
import numpy as np
import os
import time
import gdreg

In [5]:
PGEN_FILE="/n/scratch3/users/j/jz286/imp_geno/ukb_imp_chr21_v3"
BED_FILE_PATH="/n/groups/price/ldsc/reference_files/bed"
OUT_PREFIX="/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/ukb_imp_chr21_v3"

print("BED_FILE_PATH=%s" % BED_FILE_PATH)
print("OUT_PREFIX=%s" % OUT_PREFIX)

BED_FILE_LIST = []
temp_list = os.listdir(BED_FILE_PATH)
with open("bed_list.txt", "r") as f:
    for line in f:
        line = line.strip()
        if "%s.bed" % line in temp_list:
            BED_FILE_LIST.append("%s.bed" % line)
        if "%s.extend.500.bed" % line in temp_list:
            BED_FILE_LIST.append("%s.extend.500.bed" % line)
BED_FILE_LIST = sorted(BED_FILE_LIST)
print("Detected %d .bed files " % len(BED_FILE_LIST))

BED_FILE_PATH=/n/groups/price/ldsc/reference_files/bed
OUT_PREFIX=/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/ukb_imp_chr21_v3
Detected 62 .bed files 


In [11]:
# Read PGEN
dic_data = gdreg.util.read_pgen(PGEN_FILE)
CHR = dic_data["pvar"]['CHR'][0]

In [47]:
df_annot = dic_data["pvar"][["CHR", "SNP", "BP"]].copy()

for bed_file in BED_FILE_LIST:
    
    
    fpath = BED_FILE_PATH + "/" + bed_file
    df_bed = pd.read_csv(fpath, sep='\t', header=None)
    df_bed = df_bed.loc[df_bed[0]=='chr%d'%CHR]
    if df_bed.shape[1] == 3:
        df_bed[3] = 1
    df_bed.columns = ['CHR', "START", "END", "VAL"]
    df_bed["VAL"] = df_bed["VAL"].astype(np.float32)
    
    dic_bed = {}
    for START,END,VAL in zip(df_bed["START"], df_bed["END"], df_bed["VAL"]):
        temp_dic = {x:VAL for x in range(START, END)}
        dic_bed.update(temp_dic)
    
    bed_name = "AN:" + bed_file.replace('.bed', '')
    df_annot[bed_name] = [dic_bed[x] if x in dic_bed else 0 for x in df_annot['BP']]
    
    if len(df_annot[bed_name].unique())==2:
        df_annot[bed_name] = df_annot[bed_name].astype(bool)
    else:
        df_annot[bed_name] = df_annot[bed_name].astype(np.float32)
        
    print("%s : %d unique values, %s non-zeros" % (
        bed_name, len(df_annot[bed_name].unique()), (df_annot[bed_name]!=0).sum()
    ))
        
    break

# dic_data["pvar"]

AN:Backgrd_Selection_Stat : 857 unique values, 198894 non-zeros


In [48]:
gdreg.util.write_annot(df_annot, OUT_PREFIX+'.bed.annot.gz')

'CM' missing from df_annot.columns, add 'CM' column with 0


In [46]:
df_annot

,CHR,SNP,BP,Backgrd_Selection_Stat,CM
0,21,rs558824633,9413985,0.000,0
1,21,rs575710892,9418492,0.000,0
2,21,rs574994120,9419845,0.000,0
3,21,rs537592226,9419872,0.000,0
4,21,rs527627191,9491495,0.000,0
...,...,...,...,...,...
200639,21,rs546224335,48109531,0.170,0
200640,21,rs199628654,48109630,0.170,0
200641,21,rs558149838,48115770,0.149,0
200642,21,rs141010241,48116032,0.149,0


In [49]:
temp_df = pd.read_csv(
    "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/ukb_imp_chr21_v3.bed.annot.gz",
    sep='\t'
)

In [51]:
temp_df.sum(axis=0)

CHR                                                                             4213524
SNP                                   rs558824633rs575710892rs574994120rs537592226rs...
BP                                                                        6352586437292
CM                                                                                    0
AN:Backgrd_Selection_Stat                                                       21130.8
                                                            ...                        
AN:Vahedi_Tcell_TE                                                                 6581
AN:Vahedi_Tcell_TE_500bp                                                           7732
AN:WeakEnhancer_Hoffman                                                            4048
AN:WeakEnhancer_Hoffman.extend.500                                                16726
AN:alleleage                                                                4.40336e+07
Length: 66, dtype: object